# 1. Imports

In [1]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import json
import logging
import pickle
import time
import datetime

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("consumer")


config = json.load(open("config.json"))
print(json.dumps(config, indent=2))

{
  "global": {
    "kafka_bootstrap_servers": "kafka:9092",
    "kafka_topic": "test-structured-streaming",
    "kafka_consumer_group": "ss_job",
    "max_records_per_batch": 20
  }
}


In [2]:
config['global']['kafka_topic'] = 'test'

# 2. Initialize Spark with Kafak Consumer

In [3]:
# Create Spark Configuration and set application name
conf = SparkConf().setAppName("KafkaExp")

# Default pyspark installation lacks kafka consumer libraries. Install kafka-client libs manually
kafka_packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{"2.12"}:{"3.3.0"}',
    'org.apache.kafka:kafka-clients:2.8.0',
    "org.apache.hadoop:hadoop-aws:3.3.0",
    "com.google.guava:guava:21.0",
    "org.apache.httpcomponents:httpcore:4.4.8"
]

# Provide kafka jar paths to driver and executors
kafka_jar_paths = '/mnt/home/prathyush/.ivy2/jars/'.join([
    "org.apache.hadoop_hadoop-client-runtime-3.3.2.jar",
    "org.apache.kafka_kafka-clients-2.8.1.jar",
    "hadoop-aws-2.7.5.jar",
    "aws-java-sdk-core-1.12.268.jar"
])

# Connect to Spark cluster (Cluster mode instead of local mode)
conf = (conf.setMaster('spark://spark:7077')
        .set('spark.jars.packages', ','.join(kafka_packages))
        .set('spark.driver.extraClassPath', '/mnt/home/prathyush/.ivy2/jars/*')
        .set('spark.executor.extraClassPath', '/mnt/home/prathyush/.ivy2/jars/*')
        )

# Create spark context
sc = SparkContext(conf=conf)

logger.info(f"Spark Driver memory: {sc._conf.get('spark.driver.memory')}")
logger.info(f"Spark Executor memory: {sc._conf.get('spark.executor.memory')}")
logger.info(
    f'Loaded jars:\n{json.dumps((sc._jsc.sc().listJars().toList().toString().replace("List(", "").replace(")", "").split(", ")), indent=2)}')
sc.setLogLevel("ERROR")

# Create spark session
spark = SparkSession(sc)
spark.conf.set("spark.sql.parquet.compression.codec", "gzip")
spark.conf.set("mapreduce.fileoutputcommitter.marksuccessfuljobs", "false")
spark.conf.set("parquet.enable.summary-metadata", "false")

:: loading settings :: url = jar:file:/opt/bitnami/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
com.google.guava#guava added as a dependency
org.apache.httpcomponents#httpcore added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-74638790-755a-4393-ad42-1f9747b2c50b;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.0 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.

22/12/10 01:10:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
INFO:consumer:Spark Driver memory: None
INFO:consumer:Spark Executor memory: None
INFO:consumer:Loaded jars:
[
  "spark://c0d62401b111:37177/jars/org.apache.spark_spark-token-provider-kafka-0-10_2.12-3.3.0.jar",
  "spark://c0d62401b111:37177/jars/org.xerial.snappy_snappy-java-1.1.8.4.jar",
  "spark://c0d62401b111:37177/jars/org.apache.spark_spark-sql-kafka-0-10_2.12-3.3.0.jar",
  "spark://c0d62401b111:37177/jars/org.spark-project.spark_unused-1.0.0.jar",
  "spark://c0d62401b111:37177/jars/com.google.guava_guava-21.0.jar",
  "spark://c0d62401b111:37177/jars/org.apache.hadoop_hadoop-client-runtime-3.3.2.jar",
  "spark://c0d62401b111:37177/jars/org.slf4j_slf4j-api-1.7.32.jar",
  "spark://c0d62401b111:37177/jars/com.amazonaws_aws-java-sdk-bundle-1.11.563.jar",
  "spark://c0d62401b111:37177/jars/com.google.code.findbugs_jsr305-3.0.0.jar",
  "spark://c0d62401b111:

# 3. Test Kafka topic and connection

In [4]:
from confluent_kafka.admin import AdminClient

def test_kafka_connection(broker_conf:dict) -> None:
    """
    Function to test kafka connection
    :param broker_conf: Broker configuration
    :returns: None
    """
    client = AdminClient(broker_conf)
    topics = client.list_topics().topics
    if not topics:
        raise RuntimeError()
    print("Kafka Connection successful!")


broker_conf = {
    'bootstrap.servers': config["global"]["kafka_bootstrap_servers"]
}

# Test kafka connection
test_kafka_connection(broker_conf)


Kafka Connection successful!


# 4. Load Schema

In [5]:
# Load schema 
schema = pickle.load(open("schema.pkl", 'rb'))

# 5. Configure Spark-Kafka consumer options and Subscribe to Kafka Topic

In [258]:
spark.sparkContext.setLogLevel("ERROR")

# Configure spark kafka client options
spark_kafka_options = {
    "kafka.bootstrap.servers": config["global"]["kafka_bootstrap_servers"],
    "subscribe": config["global"]["kafka_topic"],
    "kafka.group.id": config["global"]["kafka_consumer_group"],
    "maxOffsetsPerTrigger": config["global"]["max_records_per_batch"],
    "startingOffsets": "earliest",
}

# Enable spark read stream
df = spark.readStream.format("kafka").option("kafka.bootstrap.servers", "kafka:9092").option("subscribe", "test").load()

# 6. Start spark structred streaming job

In [285]:
# Lambda Function for processing each batch of record
#from pyspark.sql.functions import from_json
#from pyspark.sql.functions import col
#from pyspark.sql.functions import when
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, IntegerType, ArrayType, MapType, StringType

#My UDF
def minmax(col):
    return_list = []
    if col != Null:
        year_list = list(map(int, col))
        min_year = min(year_list)
        max_year = max(year_list)
        return_list.append(min_year)
        return_list.append(max_year)
        return return_list
    else:
        return_list.append(0)
        return return_list


def process_batch(batch_df, batch_idx):
    print(f"{batch_idx} | {batch_df.count()}")
    #print(batch_idx)
    #print(batch_df.show())
    #print(batch_df.printSchema())
    # Process data and calculate
    # a. age
    #When done processing each query need to turn back into the right variable. Select all applicable columns!

    batch_df = batch_df.withColumn('Age_Years', when(col('death').isNull(), ceil(months_between(current_date(),col("birth"))/lit(12))).otherwise(ceil(months_between(col('death'), col('birth'))/lit(12))))
    # b. num_contribs
    batch_df = batch_df.withColumn("Number_of_Contributions", size(col('contribs')))
    
    # c. min_max_years
    #DataFrame[col: struct<award:string,by:string,year:string>]
    #array<struct<award:string,by:string,year:string>>
    #Was getting a weird error everytime I tried to make a UDF. Figured out how to do the same operation without using UDF. Code below shows what I would have used for udf
    batch_df = batch_df.withColumn("min", array_min(col("awards.year")))
    batch_df = batch_df.withColumn("max", array_max(col("awards.year")))
    #min_max.show()
    #min_max = min_max.withColumn("max", array_max(col("awards.year"))).show()
    batch_df = batch_df.withColumn("Award_Year_Min_Max", concat_ws(',', col('min'), col('max')))
    
    batch_df = batch_df.select("name","Age_Years", "Number_of_Contributions", "Award_Year_Min_Max")
    #There is some issue with windows/linux preventing me from saving this as a parquet file within the local file system. Attempts at changing the yml file have not worked to resolve this issue
    batch_df.write.outputMode("append").option("my_query.parquet").start()
    #batch_df.write.parquet(r"my_query.parquet")
    
    #Start of UDF code that doesnt work for some reason...
    #minmax = udf(min_max, ArrayType(StringType()))
    #min_max = min_max.select(explode(col("awards"))).show()
    #min_max.select(minmax(col("awards.year"))).show()
    
    ...

    # Select required columns -  "name", "age", "num_contribs", "min_max"
    ...

    # Save to parquet file - result.parquet 
    ...
    return batch_df

# Structred streaming query
#query = ... #



In [287]:
def p1(bdf,bidx):
    print(bdf.printSchema(), bdf.count(), bidx)
    print(bdf.show())

df = spark.readStream.format("kafka").options(**spark_kafka_options).load()
query = df.selectExpr("CAST(value AS STRING)")\
            .select(from_json(col("value"),schema).alias('data'))\
            .select('data.*')\
           .writeStream\
           .foreachBatch(process_batch)\
           .start()
print(query)

# 7. Monitor structred streaming job progress

In [289]:
# Add startup delay
time.sleep(5)
# Update Job Status

print(query.status)
while query.status['isDataAvailable'] or query.status['isTriggerActive']:
    print(query.status)
    time.sleep(5)

#query.writeStream.format("console").start()

# Stop query
query.stop()

logger.info("Structred streaming job completed successfully")

INFO:py4j.clientserver:Python Server ready to receive messages
INFO:py4j.clientserver:Received command c on object id p67


0 | 0


INFO:consumer:Structred streaming job completed successfully


{'message': 'Waiting for data to arrive', 'isDataAvailable': False, 'isTriggerActive': False}


INFO:py4j.clientserver:Received command g on object id p66
